In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
from netneurotools import cluster
from netneurotools import modularity

import networkx as nx
from sknetwork.clustering import Louvain, get_modularity
from sknetwork.data import from_edge_list

In [3]:
# DNp01 (giant fiber) to DNp11
body_ids = ["2307027729","5813024015", "1565846637", "1405231475", "1466998977", "5813023322", "1100404581", "1226887763", "1228264951", "512851433", "5813026936", "1281324958"]
DNp_ids = [int(i) for i in body_ids]

In [4]:
consensusResults = np.load("consensusResults.npy", allow_pickle=True)

In [5]:
# adj_mat = np.loadtxt("adj_mat.csv", delimiter=",")
all_connection_df = pd.read_csv("all_connection_df.csv")
dfFilt = all_connection_df[['bodyId_pre', 'bodyId_post', 'weight']] # sknetwork uses 3rd col as weight

In [7]:
# this is only needed bc Louvain's clustering assignment output is based on graph.names order
graph = from_edge_list(list(dfFilt.itertuples(index=False)), weighted=True, directed=True) # without directed=True, wrong # of elements

In [8]:
graph.adjacency, graph.names

(<5596x5596 sparse matrix of type '<class 'numpy.int64'>'
 	with 389065 stored elements in Compressed Sparse Row format>,
 array([ 326253554,  357245785,  357249472, ..., 7112615304, 7112622763,
        7112624834]))

In [9]:
# create directed, weighted networkx graph from dataframe of edges & weights
# NOTE: it seems that NetworkX conversion shuffles edge ordering (so clustering results may be different)
G = nx.from_pandas_edgelist(all_connection_df, 'bodyId_pre', 'bodyId_post', 'weight', create_using=nx.DiGraph())

In [10]:
G.number_of_nodes(), G.number_of_edges()

(5596, 389065)

In [20]:
np.where(consensus250 == 1)[0]

array([   0,    1,    2, ..., 5572, 5579, 5583])

In [23]:
# find subgraph using nodes in cluster 1
# first extract the nodes associated with cluster 1
consensus250 = consensusResults[4][1] # consensus clustering labels of each of the 5596 nodes for iteration 250
cluster1_nodeIndices = np.where(consensus250 == 1)[0] # indices of cluster1 neurons based on graph.names
cluster1_nodes = [graph.names[i] for i in cluster1_nodeIndices] # get cluster1 neuron bodyIds from graph.names

# then extract the subgraph
cluster1_subgraph = G.subgraph(cluster1_nodes)

In [24]:
cluster1_subgraph.number_of_nodes(), cluster1_subgraph.number_of_edges()

(1705, 89137)

In [26]:
# find betweenness centrality of each node in subgraph